In [1]:
from fastai.vision.all import *
import torch

In [13]:
path = Path('../../../data/public_dataset')

def label_func(file):
    status = file.split('-')[1][0]
    code = file[-8:-4]
    if code.count('0') == 4:
        syn = 0
    else:
        count = (
            (1, code.count('1')),
            (2, code.count('2')),
            (3, code.count('3')),
            (4, code.count('4')),
            (5, code.count('5'))
        )
        syn = max(count, key=itemgetter(1))[0]
    code = status + str(syn)
    covid = ['33', '03', '30', '23', '31', '32', '34']
    other = ['15', '05', '10', '25', '21', '22', '24', '01', '02', '04', '20']
    if code in covid:
        return True
    elif code in other:
        return False

In [14]:
dls = ImageDataLoaders.from_name_func(
    path=path,
    fnames=get_image_files(path),
    label_func=label_func,
    item_tfms=Resize(1025, pad_mode='zeros'),
    valid_pct=0.2,
    seed=3,
    bs=4,
    num_workers=0,
    device=torch.device('cuda')
)

In [15]:
learn = vision_learner(
    dls,
    models.resnet18,
    metrics=accuracy
)

In [16]:
learn.fine_tune(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.376133,0.335812,0.931590,27:45


epoch,train_loss,valid_loss,accuracy,time
0,0.241439,0.253197,0.932180,35:26


In [20]:
learn.predict('../../../data/public_dataset/314-10000.png')

('False', TensorBase(0), TensorBase([0.9187, 0.0813]))

In [17]:
learn.export()